In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from matplotlib import cm
import matplotlib.font_manager as fm
plt.rc('font', family=fm.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()) # for Windows OS user
import math
%matplotlib inline
import seaborn as sns
import klib
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectPercentile
from sklearn.model_selection import cross_val_score
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
import sys, warnings
if not sys.warnoptions: warnings.simplefilter("ignore")
    

from sklearn.neural_network import MLPRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import VotingRegressor

from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedKFold

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.base import ClassifierMixin
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

import os
import time
import random
import warnings; warnings.filterwarnings("ignore")
from IPython.display import Image
import pickle
from tqdm import tqdm
import platform
from itertools import combinations
from scipy.stats.mstats import gmean


import datetime
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.decomposition import PCA

In [2]:
df_x_train = pd.read_csv(os.path.abspath("../input")+'/X_train.csv', encoding='cp949')
df_x_test = pd.read_csv(os.path.abspath("../input")+'/X_test.csv', encoding='cp949')
df_y_train = pd.read_csv(os.path.abspath("../input")+'/y_train.csv').age
df_y_train_mer = pd.read_csv(os.path.abspath("../input")+'/y_train.csv')
y_train = pd.read_csv(os.path.abspath("../input")+'/y_train.csv', encoding='cp949')
IDtrain = pd.DataFrame({'custid': df_x_train.custid.unique()})
IDtest = pd.DataFrame({'custid': df_x_test.custid.unique()})

# df = pd.concat([df_x_train, df_x_test])

eda_df = pd.merge(df_x_train , df_y_train_mer, on = 'custid')

ALL = pd.read_csv(os.path.abspath("../input")+'/ALL피처추가060601.csv')

In [4]:
train_mean_vector = pd.read_csv(os.path.abspath("../input")+'/train_mean_vector.csv')
test_mean_vector = pd.read_csv(os.path.abspath("../input")+'/test_mean_vector.csv')

train_mean_vector1 = pd.read_csv(os.path.abspath("../input")+'/train_mean_vector1.csv')
test_mean_vector1 = pd.read_csv(os.path.abspath("../input")+'/test_mean_vector1.csv')

In [5]:
df = pd.read_csv(os.path.abspath("../input")+'/지평오빠feature.csv')
dft = pd.read_csv(os.path.abspath("../input")+'/지평오빠feature_te.csv')

### scaling

In [5]:
numeric_columns = df.dtypes[df.dtypes != 'object'].index.tolist()

In [6]:
# non-skew에만 처리를 하기 위해서 numeric_columns에서 skew와 non-skew로 구분
skewlist = []
numeric_columns_ns = []

for i in numeric_columns:
    if 'SKEW' in i:
        skewlist.append(i)
    else:
        numeric_columns_ns.append(i)

In [7]:
# non-skew standard scaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])
dft[numeric_columns] = scaler.transform(dft[numeric_columns])

In [8]:
trainid = IDtrain.custid.to_list()
X_train = ALL.query('custid in @trainid')

testid = IDtest.custid.to_list()
X_test = ALL.query('custid in @testid')

In [9]:
X_train = X_train.drop('custid', axis = 1 )

In [10]:
X_test = X_test.drop('custid', axis = 1 )

In [11]:
numeric_columns2 = X_train.dtypes[X_train.dtypes != 'object'].index.tolist()

In [12]:
# non-skew에만 처리를 하기 위해서 numeric_columns에서 skew와 non-skew로 구분
skewlist2 = []
numeric_columns_ns2 = []

for i in numeric_columns2:
    if 'SKEW' in i:
        skewlist2.append(i)
    else:
        numeric_columns_ns2.append(i)

In [13]:
# non-skew standard scaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train[numeric_columns2] = scaler.fit_transform(X_train[numeric_columns2])
X_test[numeric_columns2] = scaler.transform(X_test[numeric_columns2])

### feature concat

In [14]:
data = pd.concat([df[numeric_columns_ns], df[skewlist], train_mean_vector , X_train[numeric_columns_ns2], X_train[skewlist2]], axis=1)

test_mean_vector.index = dft.index
X_test.index = dft.index
data_te = pd.concat([dft[numeric_columns_ns], dft[skewlist], test_mean_vector ,  X_test[numeric_columns_ns2], X_test[skewlist2]], axis=1)

In [15]:
train_mean_vector1.columns = train_mean_vector1.columns.astype(str) + "_time"
data = pd.concat([data, train_mean_vector1], axis=1)

test_mean_vector1.index = data_te.index
test_mean_vector1.columns = test_mean_vector1.columns.astype(str) + "_time"
data_te = pd.concat([data_te, test_mean_vector1], axis=1)

In [16]:
display( data, data_te)

,custid,CUSTID_COUNT,SALES_TIME_MIN,SALES_TIME_MAX,SALES_TIME_STD,GOODCD_NUNIQUE,IMPORT_FLG_MEAN,IMPORT_FLG_SUM,IMPORT_FLG_NUNIQUE,TOT_AMT_MIN,...,90_time,91_time,92_time,93_time,94_time,95_time,96_time,97_time,98_time,99_time
0,-1.732723,-0.539868,-0.129259,0.522979,0.984931,-0.642232,2.224346,0.501115,0.475150,0.321857,...,0.256365,0.250052,-0.588806,-0.406187,0.439367,0.185279,0.181373,0.685128,0.177597,-0.129340
1,-1.732492,-0.539868,-0.950020,0.455549,2.119481,-0.642232,-0.466244,-0.488931,0.475150,-1.662230,...,0.042530,-0.419971,-0.239485,0.733378,-0.301544,0.995978,-0.650930,-0.216006,0.197088,0.797142
2,-1.732376,0.031373,-0.630835,0.610638,1.087506,0.354036,-0.914675,-0.653938,-2.104596,-0.578141,...,1.116125,-0.557568,-0.064265,0.548341,1.124598,0.094009,1.270353,-0.477902,-0.803664,0.894444
3,-1.732261,-0.750325,0.838428,-2.713662,-2.247293,-0.841486,0.318512,-0.488931,0.475150,0.335493,...,-0.111864,-0.018073,0.509619,0.201731,-1.008987,0.289059,-0.123257,-1.076202,0.199374,0.672205
4,-1.732145,0.091504,-0.696699,0.401605,1.616230,0.354036,0.010215,0.336107,0.475150,0.296404,...,-0.496266,0.330754,-0.703236,-0.781866,0.165355,0.084495,0.354446,0.697256,-0.053173,-0.477805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21582,1.732428,1.414377,-1.086814,0.408348,0.531171,2.678662,-0.395439,0.666123,0.475150,-1.375867,...,-0.410266,0.409000,-0.294169,-0.414884,-0.443909,0.351057,-0.240866,0.028628,-0.047315,0.323812
21583,1.732544,-0.299345,0.012601,0.401605,1.011767,-0.509397,-0.395439,-0.323923,0.475150,-0.010870,...,0.549509,0.009478,0.140891,0.843434,-0.524057,0.407387,-1.173958,-0.395567,0.306181,0.958077
21584,1.732659,-0.269280,0.063265,-0.825621,-0.765582,-0.243725,-0.668038,-0.488931,0.475150,0.280948,...,-0.485857,0.605453,-0.245641,-0.241068,-0.327827,0.197681,0.248651,-0.296740,-0.351735,-0.426079
21585,1.732775,-0.479737,0.331786,0.340918,-0.124821,-0.310143,-0.914675,-0.653938,-2.104596,0.193675,...,0.052067,0.453997,0.609971,0.394561,-0.917944,0.692990,-1.018453,-0.385621,-0.163522,0.667704


,custid,CUSTID_COUNT,SALES_TIME_MIN,SALES_TIME_MAX,SALES_TIME_STD,GOODCD_NUNIQUE,IMPORT_FLG_MEAN,IMPORT_FLG_SUM,IMPORT_FLG_NUNIQUE,TOT_AMT_MIN,...,90_time,91_time,92_time,93_time,94_time,95_time,96_time,97_time,98_time,99_time
0,1.733121,-0.058823,2.054371,0.388119,-1.644922,0.221200,-0.549286,-0.323923,0.475150,0.180448,...,-0.729876,0.690600,-0.534627,-0.790608,0.550694,-0.203051,0.623015,0.889328,-0.021200,-0.047212
1,1.733237,2.196075,-0.950020,0.522979,0.717249,2.080901,0.294331,3.471252,0.475150,-0.290415,...,-0.139067,0.302104,-0.191570,-0.283908,-0.355157,0.406006,-0.136448,0.012418,0.354670,0.022581
2,1.733352,0.542483,0.027800,0.522979,0.430884,0.752543,-0.389915,0.171100,0.475150,0.080493,...,-0.170458,0.789328,0.787921,0.150189,-0.091482,-0.284138,-0.354679,-0.451833,0.404842,-0.853814
3,1.733583,-0.840521,3.371641,0.340918,-2.525086,-1.040740,4.018072,-0.488931,-2.104596,0.346402,...,0.243847,1.097241,1.557096,-0.272380,0.635346,-0.351574,-1.010821,1.454252,1.535311,-0.746919
4,1.733814,-0.690194,1.532529,0.543208,-0.429491,-0.708650,-0.092551,-0.488931,0.475150,0.308220,...,0.051304,0.285071,1.738491,0.670187,0.097054,0.247967,-1.045783,-0.738167,0.132681,0.090545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14375,4.042105,-0.750325,-0.088728,-0.353611,1.230923,-0.841486,0.318512,-0.488931,0.475150,0.339584,...,-0.197278,0.738715,-1.700414,-1.095568,1.133009,-0.163281,0.482896,-0.430197,0.405807,-0.668299
14376,4.042336,-0.840521,1.345071,-2.356283,-2.525086,-1.040740,4.018072,-0.488931,-2.104596,0.559129,...,0.216434,0.823960,0.246141,1.341821,-1.019822,0.616578,-0.526211,-0.412237,-0.534695,0.445136
14377,4.042567,-0.810456,2.545814,0.003768,-1.472477,-0.974322,1.551698,-0.488931,0.475150,0.340947,...,-1.634492,0.613673,-1.001559,-1.402124,0.456310,-0.343922,1.446845,0.932912,-0.158417,-0.456060
14378,4.042683,-0.750325,1.988507,-0.131092,-1.412283,-0.907904,-0.914675,-0.653938,-2.104596,0.295948,...,0.175128,-0.184848,0.750435,-0.397384,0.299535,0.286670,-0.795192,0.655654,0.591079,-1.472640


In [17]:
y_train = pd.read_csv(os.path.abspath(os.path.abspath("../input")+'/y_train.csv', encoding='cp949').age
data.shape, data_te.shape, y_train.shape

((21587, 20391), (14380, 20391), (21587,))

In [18]:
data.columns = data.columns.astype(str)
data_te.columns = data_te.columns.astype(str)

### Modeling

In [19]:
# import re
# data = data.rename(columns = lambda x:re.sub('[^A-Za-z0-9]+', ' ', x))
# data_te = data_te.rename(columns = lambda x:re.sub('[^A-Za-z0-9]+', ' ', x))

In [20]:
data2 = data.values
data_te2 = data_te.values

In [21]:
ftr = data2[:, 1:]
target = y_train
target_log = np.log1p(target)

In [22]:
from sklearn.model_selection import train_test_split

train_x, valid_x, train_y, valid_y = train_test_split(ftr, target, test_size=0.2 , stratify = target , random_state=1000)
train_x.shape, valid_x.shape

((17269, 20390), (4318, 20390))

In [23]:
from lightgbm import LGBMRegressor

clf = LGBMRegressor(
        n_jobs=-1,
        n_estimators=1000,
        learning_rate=0.02,
        num_leaves=32,
        subsample=0.8,
        max_depth=12,
        silent=-1,
        verbose=-1
        )

clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric = 'RMSE', 
        verbose=100, early_stopping_rounds= 50)

Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 7.80921	training's l2: 60.9838	valid_1's rmse: 8.30124	valid_1's l2: 68.9106
[200]	training's rmse: 7.07975	training's l2: 50.1229	valid_1's rmse: 8.07777	valid_1's l2: 65.2504
[300]	training's rmse: 6.5901	training's l2: 43.4295	valid_1's rmse: 8.03189	valid_1's l2: 64.5112
[400]	training's rmse: 6.20279	training's l2: 38.4746	valid_1's rmse: 8.0224	valid_1's l2: 64.3589
Early stopping, best iteration is:
[434]	training's rmse: 6.08606	training's l2: 37.0401	valid_1's rmse: 8.02037	valid_1's l2: 64.3263


LGBMRegressor(learning_rate=0.02, max_depth=12, n_estimators=1000,
              num_leaves=32, silent=-1, subsample=0.8, verbose=-1)

#### Feature Selection & Cross Validation

In [24]:
ftr = data2[:, 1:]
target = y_train
target_log = np.log1p(target)

In [25]:
from sklearn.feature_selection import SelectFromModel
smf = SelectFromModel(clf, threshold='7.0*mean')
smf.fit(ftr, target)
X_new = smf.transform(ftr)
X_te_new = smf.transform(data_te2[:, 1:])
feature_idx = smf.get_support()
#feature_name = ftr.columns[feature_idx]

In [26]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(clf, X_new, target, scoring='neg_mean_squared_error', cv=5)
print('교차 검증별 정확도:', np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))
print('RMSE:', np.sqrt(-np.mean(scores)))
# mean -
# 2.0mean - 
# 3.0mean - 
# 3.5mean -       
# 4.0mean - 
# 4.5mean - 
# 5.0mean - 7.986655255475062
# 6.0mean - 
# 6.5mean - 
# 7.0mean - 7.986347806899532
# 8.0mean - 
# 9.0mean - 

교차 검증별 정확도: [-63.3759 -66.1126 -65.3234 -61.516  -62.5808]
평균 검증 정확도: -63.7818
RMSE: 7.986347806899532


In [76]:
X_new.shape, X_te_new.shape

((21587, 833), (14380, 833))

In [77]:
# pd.DataFrame(X_new).to_csv('zi_ha_sel_feature.csv', index = False )
# pd.DataFrame(X_te_new).to_csv('zi_ha_sel_feature_te.csv', index = False )

#### Hyper parameter Tuning_lgbm

In [38]:
ftr = X_new
y_train = pd.read_csv(os.path.abspath("../input")+'/y_train.csv', encoding='cp949').age
target = y_train
target_log = np.log1p(target)

In [30]:
target.shape

(21587,)

In [31]:
from sklearn.model_selection import train_test_split

train_x, valid_x, train_y, valid_y = train_test_split(ftr, target, test_size=0.3, stratify = target,  random_state=1000)
train_x.shape, valid_x.shape

((15110, 833), (6477, 833))

In [32]:
def get_rmse(model):
    pred = model.predict(valid_x)
    mse = mean_squared_error(valid_y , pred)
    rmse = np.sqrt(mse)
    print('{0} 로그 변환된 RMSE: {1}'.format(model.__class__.__name__,np.round(rmse, 3)))
    return rmse

def get_rmses(models):
    rmses = [ ]
    for model in models:
        rmse = get_rmse(model)
        rmses.append(rmse)
    return rmses



lgbm = LGBMRegressor(random_state = 1000  )
lgbm = lgbm.fit(train_x , train_y)

cat = CatBoostRegressor(random_state=1000 )
cat = cat.fit(train_x , train_y)

xgb = XGBRegressor(random_state = 1000 )
xgb.fit(train_x , train_y )

reg_ridge = Ridge(random_state = 1000)
reg_ridge.fit(train_x, train_y)


models = [lgbm, cat, xgb, reg_ridge ]
get_rmses(models)

# 8.082221840096834, 8.035338980509017, 8.555654554119082, 8.487651250878368]

Learning rate set to 0.063286
0:	learn: 10.2722687	total: 281ms	remaining: 4m 40s
1:	learn: 10.1243738	total: 394ms	remaining: 3m 16s
2:	learn: 9.9847729	total: 508ms	remaining: 2m 48s
3:	learn: 9.8494674	total: 628ms	remaining: 2m 36s
4:	learn: 9.7277755	total: 751ms	remaining: 2m 29s
5:	learn: 9.6168876	total: 865ms	remaining: 2m 23s
6:	learn: 9.5190837	total: 971ms	remaining: 2m 17s
7:	learn: 9.4303120	total: 1.09s	remaining: 2m 15s
8:	learn: 9.3388005	total: 1.21s	remaining: 2m 12s
9:	learn: 9.2571771	total: 1.32s	remaining: 2m 11s
10:	learn: 9.1784009	total: 1.44s	remaining: 2m 9s
11:	learn: 9.1141058	total: 1.57s	remaining: 2m 9s
12:	learn: 9.0518966	total: 1.7s	remaining: 2m 8s
13:	learn: 8.9961118	total: 1.81s	remaining: 2m 7s
14:	learn: 8.9404305	total: 1.92s	remaining: 2m 6s
15:	learn: 8.8880598	total: 2.03s	remaining: 2m 5s
16:	learn: 8.8453148	total: 2.15s	remaining: 2m 4s
17:	learn: 8.8014856	total: 2.26s	remaining: 2m 3s
18:	learn: 8.7627664	total: 2.39s	remaining: 2m 3s


158:	learn: 7.6385690	total: 17.9s	remaining: 1m 34s
159:	learn: 7.6352623	total: 18.1s	remaining: 1m 34s
160:	learn: 7.6298776	total: 18.2s	remaining: 1m 34s
161:	learn: 7.6268885	total: 18.3s	remaining: 1m 34s
162:	learn: 7.6229951	total: 18.4s	remaining: 1m 34s
163:	learn: 7.6175626	total: 18.5s	remaining: 1m 34s
164:	learn: 7.6142392	total: 18.6s	remaining: 1m 33s
165:	learn: 7.6097335	total: 18.7s	remaining: 1m 33s
166:	learn: 7.6072812	total: 18.8s	remaining: 1m 33s
167:	learn: 7.6060232	total: 18.9s	remaining: 1m 33s
168:	learn: 7.6009440	total: 19s	remaining: 1m 33s
169:	learn: 7.5976267	total: 19.1s	remaining: 1m 33s
170:	learn: 7.5937460	total: 19.2s	remaining: 1m 32s
171:	learn: 7.5893971	total: 19.3s	remaining: 1m 32s
172:	learn: 7.5855138	total: 19.4s	remaining: 1m 32s
173:	learn: 7.5818254	total: 19.5s	remaining: 1m 32s
174:	learn: 7.5786640	total: 19.6s	remaining: 1m 32s
175:	learn: 7.5747723	total: 19.7s	remaining: 1m 32s
176:	learn: 7.5721741	total: 19.8s	remaining: 1m

314:	learn: 7.0493721	total: 33.4s	remaining: 1m 12s
315:	learn: 7.0450058	total: 33.5s	remaining: 1m 12s
316:	learn: 7.0409368	total: 33.6s	remaining: 1m 12s
317:	learn: 7.0366382	total: 33.7s	remaining: 1m 12s
318:	learn: 7.0334652	total: 33.8s	remaining: 1m 12s
319:	learn: 7.0305815	total: 33.9s	remaining: 1m 11s
320:	learn: 7.0301526	total: 33.9s	remaining: 1m 11s
321:	learn: 7.0261896	total: 34s	remaining: 1m 11s
322:	learn: 7.0234660	total: 34.1s	remaining: 1m 11s
323:	learn: 7.0209326	total: 34.2s	remaining: 1m 11s
324:	learn: 7.0170141	total: 34.3s	remaining: 1m 11s
325:	learn: 7.0140489	total: 34.4s	remaining: 1m 11s
326:	learn: 7.0099417	total: 34.5s	remaining: 1m 11s
327:	learn: 7.0063584	total: 34.6s	remaining: 1m 10s
328:	learn: 7.0059627	total: 34.7s	remaining: 1m 10s
329:	learn: 7.0028311	total: 34.8s	remaining: 1m 10s
330:	learn: 6.9988291	total: 34.9s	remaining: 1m 10s
331:	learn: 6.9959222	total: 35s	remaining: 1m 10s
332:	learn: 6.9927883	total: 35.1s	remaining: 1m 1

473:	learn: 6.5548948	total: 49.4s	remaining: 54.8s
474:	learn: 6.5518117	total: 49.5s	remaining: 54.7s
475:	learn: 6.5481222	total: 49.6s	remaining: 54.6s
476:	learn: 6.5456546	total: 49.7s	remaining: 54.5s
477:	learn: 6.5423037	total: 49.8s	remaining: 54.4s
478:	learn: 6.5396538	total: 49.9s	remaining: 54.3s
479:	learn: 6.5380033	total: 50s	remaining: 54.2s
480:	learn: 6.5351538	total: 50.1s	remaining: 54.1s
481:	learn: 6.5312632	total: 50.3s	remaining: 54s
482:	learn: 6.5284815	total: 50.4s	remaining: 53.9s
483:	learn: 6.5243153	total: 50.5s	remaining: 53.8s
484:	learn: 6.5214120	total: 50.6s	remaining: 53.7s
485:	learn: 6.5169919	total: 50.7s	remaining: 53.6s
486:	learn: 6.5146413	total: 50.8s	remaining: 53.5s
487:	learn: 6.5114876	total: 50.9s	remaining: 53.4s
488:	learn: 6.5086311	total: 51s	remaining: 53.3s
489:	learn: 6.5061491	total: 51.1s	remaining: 53.2s
490:	learn: 6.5041020	total: 51.2s	remaining: 53.1s
491:	learn: 6.5014624	total: 51.4s	remaining: 53s
492:	learn: 6.498414

633:	learn: 6.1307557	total: 1m 6s	remaining: 38.4s
634:	learn: 6.1276342	total: 1m 6s	remaining: 38.3s
635:	learn: 6.1248088	total: 1m 6s	remaining: 38.2s
636:	learn: 6.1212621	total: 1m 6s	remaining: 38.1s
637:	learn: 6.1191905	total: 1m 6s	remaining: 38s
638:	learn: 6.1189662	total: 1m 7s	remaining: 37.9s
639:	learn: 6.1161239	total: 1m 7s	remaining: 37.8s
640:	learn: 6.1140359	total: 1m 7s	remaining: 37.7s
641:	learn: 6.1113350	total: 1m 7s	remaining: 37.6s
642:	learn: 6.1097280	total: 1m 7s	remaining: 37.5s
643:	learn: 6.1067085	total: 1m 7s	remaining: 37.4s
644:	learn: 6.1034050	total: 1m 7s	remaining: 37.3s
645:	learn: 6.0985909	total: 1m 7s	remaining: 37.2s
646:	learn: 6.0941974	total: 1m 7s	remaining: 37.1s
647:	learn: 6.0918354	total: 1m 8s	remaining: 37s
648:	learn: 6.0898109	total: 1m 8s	remaining: 36.9s
649:	learn: 6.0872011	total: 1m 8s	remaining: 36.8s
650:	learn: 6.0834533	total: 1m 8s	remaining: 36.7s
651:	learn: 6.0808624	total: 1m 8s	remaining: 36.6s
652:	learn: 6.07

789:	learn: 5.7480012	total: 1m 23s	remaining: 22.1s
790:	learn: 5.7458688	total: 1m 23s	remaining: 22s
791:	learn: 5.7439989	total: 1m 23s	remaining: 21.9s
792:	learn: 5.7410715	total: 1m 23s	remaining: 21.8s
793:	learn: 5.7392818	total: 1m 23s	remaining: 21.7s
794:	learn: 5.7371238	total: 1m 23s	remaining: 21.6s
795:	learn: 5.7348656	total: 1m 23s	remaining: 21.5s
796:	learn: 5.7323028	total: 1m 23s	remaining: 21.4s
797:	learn: 5.7295349	total: 1m 24s	remaining: 21.3s
798:	learn: 5.7281079	total: 1m 24s	remaining: 21.2s
799:	learn: 5.7254232	total: 1m 24s	remaining: 21.1s
800:	learn: 5.7216037	total: 1m 24s	remaining: 21s
801:	learn: 5.7191971	total: 1m 24s	remaining: 20.9s
802:	learn: 5.7155905	total: 1m 24s	remaining: 20.8s
803:	learn: 5.7132690	total: 1m 24s	remaining: 20.7s
804:	learn: 5.7107092	total: 1m 24s	remaining: 20.6s
805:	learn: 5.7083244	total: 1m 24s	remaining: 20.4s
806:	learn: 5.7062749	total: 1m 25s	remaining: 20.3s
807:	learn: 5.7036144	total: 1m 25s	remaining: 20.

945:	learn: 5.3797756	total: 1m 39s	remaining: 5.68s
946:	learn: 5.3772058	total: 1m 39s	remaining: 5.58s
947:	learn: 5.3740586	total: 1m 39s	remaining: 5.47s
948:	learn: 5.3726741	total: 1m 39s	remaining: 5.37s
949:	learn: 5.3708495	total: 1m 39s	remaining: 5.26s
950:	learn: 5.3689764	total: 1m 40s	remaining: 5.16s
951:	learn: 5.3666822	total: 1m 40s	remaining: 5.05s
952:	learn: 5.3652571	total: 1m 40s	remaining: 4.95s
953:	learn: 5.3633997	total: 1m 40s	remaining: 4.84s
954:	learn: 5.3612523	total: 1m 40s	remaining: 4.73s
955:	learn: 5.3591457	total: 1m 40s	remaining: 4.63s
956:	learn: 5.3565540	total: 1m 40s	remaining: 4.52s
957:	learn: 5.3530074	total: 1m 40s	remaining: 4.42s
958:	learn: 5.3507986	total: 1m 40s	remaining: 4.31s
959:	learn: 5.3484256	total: 1m 40s	remaining: 4.21s
960:	learn: 5.3459785	total: 1m 41s	remaining: 4.1s
961:	learn: 5.3432129	total: 1m 41s	remaining: 3.99s
962:	learn: 5.3401960	total: 1m 41s	remaining: 3.89s
963:	learn: 5.3379702	total: 1m 41s	remaining: 

[8.082221840096834, 8.035338980509017, 8.555654554119082, 8.487651250878368]

In [46]:
from bayes_opt import BayesianOptimization

In [47]:
bayesian_params = {
    'max_depth':(8, 16),
    'num_leaves':(24, 64),
    'min_child_samples':(10, 200),
    'min_child_weight':(1, 50),
    'subsample':(0.5, 1),
    'colsample_bytree':(0.5, 1),
    'max_bin':(10, 500),
    'reg_lambda':(0.001, 10),
    'reg_alpha':(0.01, 50)
}

In [48]:
def lgb_rmse_eval(max_depth, num_leaves, min_child_samples, min_child_weight, subsample, 
                colsample_bytree, max_bin, reg_lambda, reg_alpha):
    
    params = {
        "n_estimators":2000, 
        "learning_rate":0.02,
        'max_depth':int(round(max_depth)),
        'num_leaves':int(round(num_leaves)),
        'min_child_samples': int(round(min_child_samples)),
        'min_child_weight': int(round(min_child_weight)),
        'subsample':max(min(subsample, 1), 0),
        'colsample_bytree':max(min(colsample_bytree, 1), 0),
        'reg_lambda': max(reg_lambda,0),
        'reg_alpha': max(reg_alpha, 0)
    }
    
    lgb_model = LGBMRegressor(**params)
    lgb_model.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric= 'RMSE', verbose= 100, 
                early_stopping_rounds= 100)
    valid_pred = lgb_model.predict(valid_x)
    RMSE = np.sqrt(mean_squared_error(valid_y, valid_pred))
    
    return RMSE

In [49]:
lgbBO = BayesianOptimization(f=lgb_rmse_eval, pbounds=bayesian_params, random_state=1000)
lgbBO.maximize(init_points=5, n_iter=25)

|   iter    |  target   | colsam... |  max_bin  | max_depth | min_ch... | min_ch... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 7.82935	training's l2: 61.2987	valid_1's rmse: 8.30778	valid_1's l2: 69.0192
[200]	training's rmse: 7.15116	training's l2: 51.1391	valid_1's rmse: 8.08957	valid_1's l2: 65.4412
[300]	training's rmse: 6.69638	training's l2: 44.8415	valid_1's rmse: 8.03895	valid_1's l2: 64.6247
[400]	training's rmse: 6.33075	training's l2: 40.0784	valid_1's rmse: 8.01757	valid_1's l2: 64.2814
[500]	training's rmse: 6.01426	training's l2: 36.1713	valid_1's rmse: 8.0095	valid_1's l2: 64.1521
[600]	training's rmse: 5.73295	training's l2: 32.8667	valid_1's rmse: 8.00917	valid_1's l2: 64.1467
Early stopping, best iteration is:
[537]	training's rmse: 5.90722	train

Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 7.75464	training's l2: 60.1344	valid_1's rmse: 8.2913	valid_1's l2: 68.7456
[200]	training's rmse: 7.03408	training's l2: 49.4782	valid_1's rmse: 8.07276	valid_1's l2: 65.1694
[300]	training's rmse: 6.54866	training's l2: 42.885	valid_1's rmse: 8.02173	valid_1's l2: 64.3481
[400]	training's rmse: 6.14817	training's l2: 37.7999	valid_1's rmse: 7.99908	valid_1's l2: 63.9853
[500]	training's rmse: 5.80486	training's l2: 33.6964	valid_1's rmse: 7.99507	valid_1's l2: 63.9212
[600]	training's rmse: 5.50386	training's l2: 30.2925	valid_1's rmse: 7.9948	valid_1's l2: 63.9169
Early stopping, best iteration is:
[539]	training's rmse: 5.683	training's l2: 32.2964	valid_1's rmse: 7.99414	valid_1's l2: 63.9062
|  8        |  7.994    |  0.7675   |  132.0    |  14.3     |  40.39    |  44.76    |  35.85    |  33.16    |  4.179    |  0.9905   |
Training until validation scores don't improve for 100 rounds
[100]	traini

[500]	training's rmse: 5.21249	training's l2: 27.1701	valid_1's rmse: 7.98705	valid_1's l2: 63.7929
[600]	training's rmse: 4.83784	training's l2: 23.4047	valid_1's rmse: 7.98179	valid_1's l2: 63.709
[700]	training's rmse: 4.49471	training's l2: 20.2024	valid_1's rmse: 7.9857	valid_1's l2: 63.7715
Early stopping, best iteration is:
[618]	training's rmse: 4.77329	training's l2: 22.7843	valid_1's rmse: 7.9802	valid_1's l2: 63.6836
|  15       |  7.98     |  0.6659   |  318.9    |  14.7     |  89.29    |  7.679    |  54.4     |  41.98    |  4.397    |  0.6464   |
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 7.41937	training's l2: 55.047	valid_1's rmse: 8.26974	valid_1's l2: 68.3886
[200]	training's rmse: 6.52574	training's l2: 42.5853	valid_1's rmse: 8.0731	valid_1's l2: 65.175
[300]	training's rmse: 5.92428	training's l2: 35.0971	valid_1's rmse: 8.03735	valid_1's l2: 64.599
[400]	training's rmse: 5.44183	training's l2: 29.6135	valid_1's rmse: 8.0193

[600]	training's rmse: 5.39813	training's l2: 29.1398	valid_1's rmse: 7.98598	valid_1's l2: 63.7759
Early stopping, best iteration is:
[571]	training's rmse: 5.48647	training's l2: 30.1013	valid_1's rmse: 7.98456	valid_1's l2: 63.7532
|  23       |  7.985    |  0.819    |  33.68    |  14.31    |  81.77    |  30.16    |  39.27    |  19.79    |  7.493    |  0.6173   |
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 7.60431	training's l2: 57.8256	valid_1's rmse: 8.27664	valid_1's l2: 68.5028
[200]	training's rmse: 6.7916	training's l2: 46.1258	valid_1's rmse: 8.06245	valid_1's l2: 65.003
[300]	training's rmse: 6.23353	training's l2: 38.8568	valid_1's rmse: 8.01667	valid_1's l2: 64.267
[400]	training's rmse: 5.77155	training's l2: 33.3108	valid_1's rmse: 8.00322	valid_1's l2: 64.0515
[500]	training's rmse: 5.376	training's l2: 28.9013	valid_1's rmse: 7.99383	valid_1's l2: 63.9013
[600]	training's rmse: 5.02397	training's l2: 25.2403	valid_1's rmse: 7.99

In [50]:
lgbBO.res

[{'target': 8.006492932036826,
  'params': {'colsample_bytree': 0.8267947927323047,
   'max_bin': 66.35340213095881,
   'max_depth': 15.602262914792195,
   'min_child_samples': 101.61636627131966,
   'min_child_weight': 43.75125222391973,
   'num_leaves': 32.4933072369088,
   'reg_alpha': 2.045074142206765,
   'reg_lambda': 3.9725474189957124,
   'subsample': 0.61656609867419}},
 {'target': 8.014310481643134,
  'params': {'colsample_bytree': 0.9208703621265308,
   'max_bin': 111.47034874950597,
   'max_depth': 13.93975626865927,
   'min_child_samples': 84.50928428367985,
   'min_child_weight': 9.930569477666833,
   'num_leaves': 53.7415765836856,
   'reg_alpha': 3.488408227688028,
   'reg_lambda': 8.853486706603126,
   'subsample': 0.9763221996107709}},
 {'target': 7.998092920982338,
  'params': {'colsample_bytree': 0.9655717173387555,
   'max_bin': 213.56116699200484,
   'max_depth': 8.23185327509497,
   'min_child_samples': 196.58522207365755,
   'min_child_weight': 17.64224649812914

In [51]:
# dictionary에 있는 target값을 모두 추출
target_list = []
for result in lgbBO.res:
    target = result['target']
    target_list.append(target)
print(target_list)
# 가장 큰 target 값을 가지는 순번(index)를 추출
print('maximum target index:', np.argmin(np.array(target_list)))

[8.006492932036826, 8.014310481643134, 7.998092920982338, 7.992138851168561, 8.004424567294615, 8.001867692720612, 7.991538986549537, 7.994137335015508, 7.98921599431001, 8.014749385657526, 7.99672422875902, 7.986388461450105, 7.97863800882115, 8.007549383603234, 7.980198319418749, 8.008850627579472, 7.9999683736445135, 8.001084702668269, 8.00399353959004, 7.992143567446837, 8.002634994010915, 7.994321259369537, 7.984557215921472, 7.992411960303955, 8.011117912147114, 7.995241113707441, 7.990347875112145, 8.004099657215061, 7.998469381383123, 7.992004901349113]
maximum target index: 12


In [52]:
# 가장 큰 target값을 가지는 index값을 기준으로 res에서 해당 parameter 추출. 
max_dict = lgbBO.res[np.argmin(np.array(target_list))]
print(max_dict)

{'target': 7.97863800882115, 'params': {'colsample_bytree': 0.579276254301057, 'max_bin': 255.18517768650602, 'max_depth': 13.550001654403957, 'min_child_samples': 12.505066102402521, 'min_child_weight': 29.42773011515404, 'num_leaves': 39.820389083540235, 'reg_alpha': 33.960034049410424, 'reg_lambda': 5.65260345331489, 'subsample': 0.8581227529538189}}


In [53]:
ftr = DataFrame(X_new)
y_train = pd.read_csv(os.path.abspath("../input")+'/y_train.csv', encoding='cp949').age
target = y_train
target_log = np.log1p(target)

In [54]:
from sklearn.model_selection import KFold

def train_apps_all_with_oof(ftr, target, nfolds=5):
    ftr = ftr
    target = target

    # nfolds 개의 cross validatin fold set을 가지는 KFold 생성 
    folds = KFold(n_splits=nfolds, shuffle=True, random_state=0)
    
    # Out of Folds로 학습된 모델의 validation set을 예측하여 결과 확률을 담을 array 생성.
    # validation set가 n_split갯수만큼 있으므로 크기는 ftr_app의 크기가 되어야 함. 
    oof_preds = np.zeros((ftr.shape[0],))  
    
    # Ouf of Folds로 학습된 모델의 test dataset을 예측하여 결과 확률을 담을 array 생성. 
    test_preds = np.zeros(((X_te_new.shape[0],)))
    
    # n_estimators를 4000까지 확대. 
    clf = LGBMRegressor(
                nthread=4,
                n_estimators=4000,
                learning_rate=0.01,
                max_depth=11,
                num_leaves=35,
                colsample_bytree=0.8385,
                subsample=0.760,
                max_bin=251,
                reg_alpha=33.817,
                reg_lambda=8.207,
                min_child_weight=26,
                min_child_samples=12,
                silent=-1,
                verbose=-1,
                )

    # nfolds 번 cross validation Iteration 반복하면서 OOF 방식으로 학습 및 테스트 데이터 예측
    for fold_idx, (train_idx, valid_idx) in enumerate(folds.split(ftr)):
        print('##### iteration ', fold_idx, ' 시작')
        # 학습용 데이터 세트의 인덱스와 검증용 데이터 세트의 인덱스 추출하여 이를 기반으로 학습/검증 데이터 추출
        train_x = ftr.iloc[train_idx, :]
        train_y = target.iloc[train_idx]
        valid_x = ftr.iloc[valid_idx, :]
        valid_y = target.iloc[valid_idx]
        
        # 추출된 학습/검증 데이터 세트로 모델 학습. early_stopping은 200으로 증가. 
        clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric= 'RMSE', verbose= 200, 
                early_stopping_rounds= 200)
        # 검증 데이터 세트로 예측된 확률 저장. 사용되지는 않음. 
        #oof_preds[valid_idx] = clf.predict(valid_x, num_iteration=clf.best_iteration_)       
        # 학습된 모델로 테스트 데이터 세트에 예측 확률 계산. 
        # nfolds 번 반복 실행하므로 평균 확률을 구하기 위해 개별 수행시 마다 수행 횟수로 나눈 확률을 추후에 더해서 최종 평균 확률 계산. 
        test_preds += clf.predict(X_te_new, num_iteration=clf.best_iteration_)/folds.n_splits
        
        
    return clf, test_preds

In [55]:
clf, test_preds = train_apps_all_with_oof(ftr, target, nfolds=5)

##### iteration  0  시작
[LightGBM] [Warning] num_threads is set with n_jobs=-1, nthread=4 will be ignored. Current value: num_threads=-1
Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 7.8261	training's l2: 61.2478	valid_1's rmse: 8.35968	valid_1's l2: 69.8842
[400]	training's rmse: 7.16539	training's l2: 51.3428	valid_1's rmse: 8.1451	valid_1's l2: 66.3427
[600]	training's rmse: 6.73373	training's l2: 45.3431	valid_1's rmse: 8.08987	valid_1's l2: 65.4461
[800]	training's rmse: 6.37603	training's l2: 40.6537	valid_1's rmse: 8.06218	valid_1's l2: 64.9988
[1000]	training's rmse: 6.06347	training's l2: 36.7657	valid_1's rmse: 8.05183	valid_1's l2: 64.832
[1200]	training's rmse: 5.78837	training's l2: 33.5052	valid_1's rmse: 8.04658	valid_1's l2: 64.7475
[1400]	training's rmse: 5.5382	training's l2: 30.6717	valid_1's rmse: 8.0442	valid_1's l2: 64.7092
[1600]	training's rmse: 5.30689	training's l2: 28.1631	valid_1's rmse: 8.04275	valid_1's l2: 64.6858
[18

In [56]:
test_preds

array([42.02959286, 41.67688766, 25.54189981, ..., 36.42954787,
       33.38099716, 28.72781476])

In [57]:
IDtest['age'] = test_preds ; sub = IDtest

In [58]:
sub.to_csv('submissions_0613_zi_ha_scaled_lgbm_tun.csv', index=False)